In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
# [2]번줄 코드 해석 :
# MNIST라는 숫자 데이터셋을 다운받기 위하여, torchvison 이라는 라이브러를 미리 다운로드 한다.
# 가지고 있는 데이터의 순서를 섞거나 원하는 비율로 나누거나 하는 데이터를 전처리르 위해 DataLoader를 선언한다.

In [ ]:
batch_size = 256

learning_rate = 0.0002

num_epoch = 10

In [ ]:
# CNN에서 batch_size는 한번에 학습하는 이미지의 수이다. 즉, MNIST는 6만장의 데이터가 있고,
# 이걸 한장한장씩 학습하는 것이 아닌,ㄴ 256개씩 묶어서 진행하겠다는 뜻이다. (256개가 아니어도 된다.)
# 6만장의 사진을 학습하므로, learning rate는 조금 낮은 값으로 잡는것이 발산할 수 있는 가능성을 낮추어준다.

# 데이터의 사이즈가 큰 관계로 epoch는 10번만 해준다.

In [ ]:
mnist_train = dset.MNIST("./",train=True, transform = transforms.ToTensor(), target_transform=None, download = True)

mnist_test = dset.MNIST("./", train=False, transform = transforms.ToTensor(), target_transform=None, download = True)

# torchvision.datasets 라이브러리에서 MNIST 데이터를 받아오는 코드

In [ ]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

# 받아온 데이터를 학습하기 위해 나누어준다.
# bath_size 선언, shuffle : 데이터를 무작위로 섞을 때
# num_workers : 데이터를 묶을 때 사용하는 프로세스 갯수
# drop_last : 묶고 남은 자투리 데이터들을 버릴지 말지

In [ ]:
class CNN(nn.Module):

    #C++에서 사용되는 Class 선언(파이썬 : 객체지향 언어)

    def __init__(self) :

        super(CNN,self).__init__() #Super class로 지금 작성하고있는 클래스 자체를 초기화하기 위함

        self.layer = nn.Sequential(

            nn.Conv2d(1,16,5),

            nn.ReLU(),

            nn.Conv2d(16,32,5),

            nn.ReLU(),

            nn.MaxPool2d(2,2),

            nn.Conv2d(32,64,5),

            nn.ReLU(),

            nn.MaxPool2d(2,2)

        )

        #Conv2d : Convolution Filtering이라는 Signal Processing적인 방법으로 이미지를 처리 하는것으로,

        #nn.Conv2d(1,16,5)는 1개필터짜리 입력(28x28 해상도의 이미지, default filter 갯수 = 1)을 받아 16개의 필터로 size 5의 Kernel(Filtering)을 하는것입니다.

        #기본적으로 CNN은 신호/영상처리에 대한 기본적인 이해가 있어야합니다.

        #Kernel size가 5인경우, Convoltuion을 하게 되면 4개의 pixel이 사라지게 되어(28x28)의 input 이미지가 (24x24)가 됩니다.

        #이런식으로 이미지의 사이즈를 줄여가며 강한 특징만을 추려나가는게 CNN입니다.



        #MaxPooling을 중간중간 섞어줌으로써, Convolution보다 더욱 강하게 Feature들을 뽑아내줍니다.

        self.fc_layer = nn.Sequential(

            nn.Linear(64*3*3,100),

            nn.ReLU(),

            nn.Linear(100,10)

        )

        #self.layer : CNN이 끝난 이후, 최종적으로 나오는 결과물은 [batch_size,64,3,3]입니다.

        #즉, 256개의 이미지 묶음씩 64개의 필터, (3x3)의 이미지가 남게 되는것으로, pixel갯수로 따지면 64*3*3이 나오게 되는것입니다.

        #따라서, 64*3*3의 결과값을 nn.Linear(100,10)을 통해 최종적으로 10개의 값이 나오게하는데

        #이 10개의 값이 내가 넣은 이미지가 0~9(10개)중 어떤것일지에 대한 각각의 확률입니다.

    def forward(self,x):

        out = self.layer(x)

        out = out.view(batch_size, -1)

        out = self.fc_layer(out)

        return out

        #CNN함수의 전체적인 그림으로, Conv2d -> Linear Regression -> 추정 입니다.



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  #이부분은 굳이 안해주셔도 됩니다. GPU를 사용할 수 없는경우 CPU를 쓰겠다는 것으로, 이부분을 주석처리하고

  # model = CNN()로만 해주셔도 됩니다.

model = CNN().to(device)

loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

#Cross Entropy Loss function, Adam optimizer



loss_arr = []

for i in range(num_epoch):

    for j,[image,label] in enumerate(train_loader):

        x = image.to(device)

        #mnist 학습용 data를 불러옵니다.(28x28)

        y_ = label.to(device)

        #각각의 data들이 0~9중 어떤숫자인지도 불러옵니다.

        optimizer.zero_grad()

        #optimizer 초기화

        output = model.forward(x)

        #학습용 데이터로 CNN 실시

        loss = loss_func(output,y_)

        #학습해서 추정해낸 값과, 실제 라벨된 값 비교

        loss.backward()

        #오차만큼 다시 Back Propagation 시행

        optimizer.step()

        #Back Propagation시 ADAM optimizer 매 Step마다 시행

        if j % 1000 == 0 :

            print(loss)

            loss_arr.append(loss.cpu().detach().numpy())



correct = 0

total = 0

with torch.no_grad():

    for image,label in test_loader :

        x = image.to(device)

        y_ = label.to(device)



        output = model.forward(x)

        _,output_index = torch.max(output,1)



        total += label.size(0)

        correct += (output_index == y_).sum().float()



    print("Accuracy of Test Data : {}".format(100*correct/total))

tensor(2.3037, grad_fn=<NllLossBackward0>)
tensor(0.2792, grad_fn=<NllLossBackward0>)
tensor(0.1016, grad_fn=<NllLossBackward0>)
tensor(0.1267, grad_fn=<NllLossBackward0>)
tensor(0.0552, grad_fn=<NllLossBackward0>)
tensor(0.0928, grad_fn=<NllLossBackward0>)
tensor(0.0588, grad_fn=<NllLossBackward0>)
tensor(0.0759, grad_fn=<NllLossBackward0>)
tensor(0.0192, grad_fn=<NllLossBackward0>)
tensor(0.0562, grad_fn=<NllLossBackward0>)
Accuracy of Test Data : 98.6278076171875


In [ ]:
# 해석 : Train Data로 학습시키고, Test Dataset으로 검증하면,
# 약 98.66%의 정확도로 사진의 숫자를 추정하는 것을 확인 할 수 있다.